In [1]:
list.of.packages <- c("tidyverse","dplyr","ggplot2")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, repos = "http://cran.us.r-project.org")

invisible(lapply(list.of.packages, library, character.only = TRUE))

select <- dplyr::select

options(repr.matrix.max.rows=50, repr.matrix.max.cols=500) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.3
✔ tibble  3.1.0     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



### Download topic classification database

In [2]:
df_class <- read_csv("../3_Topic_modeling/output/database_seuil_0.02.csv")
sprintf("%i x %i dataframe", nrow(df_class), ncol(df_class))

Parsed with column specification:
cols(
  .default = col_double(),
  ISO_3 = col_character(),
  Country = col_character(),
  Region = col_character(),
  title = col_character(),
  authors = col_character(),
  source = col_character(),
  doi = col_character(),
  abstract = col_character(),
  author_keywords = col_character(),
  model = col_character(),
  scopus_number = col_character(),
  WOS_number = col_character(),
  stem_abstract = col_character()
)

See spec(...) for full column specifications.



[1] "4691 x 55 dataframe"

In [3]:
df_class <- df_class %>% 
                    select(title, 16:54)

### Table with final topics names

In [4]:
topics_table <- read_csv("../4_Manual_treatment_topic_table/ouput/Table_topics.csv")
sprintf("%i x %i dataframe", nrow(topics_table), ncol(topics_table))

Parsed with column specification:
cols(
  Index = col_double(),
  Name = col_character(),
  Terms = col_character(),
  Category = col_character(),
  Sector = col_character(),
  `Relevant IPCC WG3 chapter` = col_character(),
  T0.02 = col_double(),
  T0.02Title = col_double()
)



[1] "39 x 8 dataframe"

#### List of topics with initial apparition order

In [5]:
topics_list <- as.character(topics_table$Name)

In [6]:
df_class <- df_class %>% 
                rename_if(is.numeric, ~ topics_list) 

### Add topic classification to the database with one row for each publication-country-model association

In [7]:
path_multi <- "../2_Treatment_database/output/database_multi_rows_each_paper.csv"
df_multi <- read_csv(path_multi)
sprintf("%i x %i dataframe", nrow(df_multi), ncol(df_multi))

Parsed with column specification:
cols(
  ISO_3 = col_character(),
  Country = col_character(),
  Region = col_character(),
  title = col_character(),
  authors = col_character(),
  source = col_character(),
  doi = col_character(),
  abstract = col_character(),
  author_keywords = col_character(),
  publication_year = col_double(),
  horizon_year = col_double(),
  model = col_character(),
  scopus_number = col_character(),
  WOS_number = col_character()
)



[1] "4996 x 14 dataframe"

In [8]:
df_multi <- df_multi %>%
                left_join(df_class,by="title")
sprintf("%i x %i dataframe", nrow(df_multi), ncol(df_multi))

[1] "4996 x 53 dataframe"

In [9]:
write_csv(df_multi,"./output/database_multi_rows_each_paper.csv")

### Create the database with one row for each publication-country association
(If numerous models are related to a paper, the column *model* presents the first one by alphaetical order)

In [10]:
df_multi_one_country <- df_multi %>%
                                group_by(Country,title)%>%
                                summarize_all(list(first))
sprintf("%i x %i dataframe", nrow(df_multi_one_country), ncol(df_multi_one_country))

[1] "4884 x 53 dataframe"

In [11]:
write_csv(df_multi_one_country,"./output/database_multi_rows_each_paper_one_per_country.csv")

### Add classification to the database with one row for each publication 
(if numerous countries or models are related to a paper, the columns "Country" and "model"
present the first one by alphabetical order)

In [12]:
path_one <- "../2_Treatment_database/output/database_one_row_each_paper.csv"
df_one <- read_csv(path_one)
sprintf("%i x %i dataframe", nrow(df_one), ncol(df_one))

Parsed with column specification:
cols(
  ISO_3 = col_character(),
  Country = col_character(),
  Region = col_character(),
  title = col_character(),
  authors = col_character(),
  source = col_character(),
  doi = col_character(),
  abstract = col_character(),
  author_keywords = col_character(),
  publication_year = col_double(),
  horizon_year = col_double(),
  model = col_character(),
  scopus_number = col_character(),
  WOS_number = col_character()
)



[1] "4691 x 14 dataframe"

In [13]:
df_one <- df_one %>%
                left_join(df_class,by="title")
sprintf("%i x %i dataframe", nrow(df_one), ncol(df_one))

[1] "4691 x 53 dataframe"

In [14]:
write_csv(df_one,"./output/database_one_row_each_paper.csv")